In [1]:
# Import the libraries we need
import pandas as pd
import re
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import datetime

In [8]:
# set url for understat
plyrs_xg_url = 'https://fbref.com/en/comps/9/stats/Premier-League-Stats'

# download the webpage
plyrs_xg_data = requests.get(plyrs_xg_url)

# get the html code for the webpage
plyrs_xg_html = plyrs_xg_data.content

# parse the html using bs4
soup = BeautifulSoup(plyrs_xg_html, 'lxml')

print(soup.title)

<title>Premier League Player Stats | FBref.com</title>


In [9]:
#set up the selenium driver
options = webdriver.ChromeOptions()

# run without opening the browser
options.add_argument('headless')

driver = webdriver.Chrome(executable_path = '/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/chrome/chromedriver', options = options)

# navigate to the page url
driver.get(plyrs_xg_url)

# get the html code from the webpage
soup = BeautifulSoup(driver.page_source, 'lxml')

print(soup.title)

<title>Premier League Player Stats | FBref.com</title>


In [26]:
# Get the table headers using 3 chained find operations
# 1. Find the div containing the table (div class = chemp jTable)
# 2. Find the table within that div
# 3. Find all 'th' elements where class = sort
headers = soup.find('div', attrs={'class':'overthrow table_container'}).find('table').find_all('th',attrs={'class':'sort'})
 
headers

[<th aria-label="Squad" class="poptip sort_default_asc center" data-stat="squad" scope="col">Squad</th>,
 <th aria-label="Number of Players used in Games" class="poptip center" data-stat="players_used" data-tip="Number of Players used in Games" scope="col"># Pl</th>,
 <th aria-label="PossessionCalculated as the percentage of passes attempted" class="poptip center" data-stat="possession" data-tip="&lt;strong&gt;Possession&lt;/strong&gt;&lt;br&gt;Calculated as the percentage of passes attempted" scope="col">Poss</th>,
 <th aria-label="Matches Played" class="poptip center group_start" data-over-header="Playing Time" data-stat="games" data-tip="&lt;strong&gt;Matches Played&lt;/strong&gt;&lt;br&gt;Matches Played by the player or squad" scope="col">MP</th>,
 <th aria-label="Starts" class="poptip center" data-over-header="Playing Time" data-stat="games_starts" data-tip="&lt;strong&gt;Starts&lt;/strong&gt;&lt;br&gt;Game or games started by player" scope="col">Starts</th>,
 <th aria-label="Minu

In [27]:
# iterate over headers, get the text from each item, and add the results to headers_list
headers_list=[]
for header in headers:
    headers_list.append(header.get_text(strip=True))
print(headers_list)

['Squad', '# Pl', 'Poss', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'CrdY', 'CrdR', 'Gls', 'Ast', 'G+A', 'G-PK', 'G+A-PK', 'xG', 'npxG', 'xA', 'xG', 'xA', 'xG+xA', 'npxG', 'npxG+xA']


In [28]:
body = soup.find('div', attrs={'class':'overthrow table_container'}).table.tbody

#create a master list for row data
all_rows_list = []

#for each row in the table body
for tr in body.find_all('tr'):
    #get data from each cell in the row
    row = tr.find_all('td')
    #create list to save current row data to
    current_row = []
    # for each item in the row variable
    for item in row:
    #add the text data to the current_row list
        current_row.append(item.get_text(strip=True))
    
    #add the current row data to the master list
    all_rows_list.append(current_row)
        
# create a dataframe where the rows = all_rows_list and columns = headers_list
plyrs_xg_df = pd.DataFrame(all_rows_list, columns=headers_list)
plyrs_xg_df

ValueError: 25 columns passed, passed data had 24 columns